In [8]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

**Boring Imports**

In [2]:
%%capture
import io
import locale
import numpy as np
import pandas as pd
import urllib2
%matplotlib

**Grab the Raw CSV Files from the Server**

In [15]:
%%capture
locale.setlocale(locale.LC_ALL, '')

base_csv_url = 'ftp://ftpx.sandag.org/sandag/pub/tnet/sr52/base_case_mandatory_tours.csv'
alt_csv_url = 'ftp://ftpx.sandag.org/sandag/pub/tnet/sr52/alt_case_mandatory_tours.csv'

base_csv_req = urllib2.Request(base_csv_url)
base_csv = urllib2.urlopen(base_csv_req).read()

alt_csv_req = urllib2.Request(alt_csv_url)
alt_csv = urllib2.urlopen(alt_csv_req).read()

base_df = pd.read_csv(io.StringIO(base_csv.decode('utf-8')))
alt_df = pd.read_csv(io.StringIO(alt_csv.decode('utf-8')))


base_df['home_msa_name'] = ''
alt_df['home_msa_name'] = ''

alt_df.ix[alt_df.home_msa == 0, 'home_msa_name'] = 'Central'
alt_df.ix[alt_df.home_msa == 1, 'home_msa_name'] = 'North City'
alt_df.ix[alt_df.home_msa == 2, 'home_msa_name'] = 'South Suburban'
alt_df.ix[alt_df.home_msa == 3, 'home_msa_name'] = 'East Suburban'
alt_df.ix[alt_df.home_msa == 4, 'home_msa_name'] = 'North County West'
alt_df.ix[alt_df.home_msa == 5, 'home_msa_name'] = 'North County East'
alt_df.ix[alt_df.home_msa == 6, 'home_msa_name'] = 'East County'

base_df.ix[base_df.home_msa == 0, 'home_msa_name'] = 'Central'
base_df.ix[base_df.home_msa == 1, 'home_msa_name'] = 'North City'
base_df.ix[base_df.home_msa == 2, 'home_msa_name'] = 'South Suburban'
base_df.ix[base_df.home_msa == 3, 'home_msa_name'] = 'East Suburban'
base_df.ix[base_df.home_msa == 4, 'home_msa_name'] = 'North County West'
base_df.ix[base_df.home_msa == 5, 'home_msa_name'] = 'North County East'
base_df.ix[base_df.home_msa == 6, 'home_msa_name'] = 'East County'

In [38]:
#5 days / week * 50 weeks
annualization_factor = 250

#https://www.transportation.gov/regulations/economic-values-used-in-analysis
value_of_time = 12.5

In [39]:
alt_msa_summ_df = alt_df.groupby(['home_msa_name'])['tour_time'].agg(['mean', 'count'])
base_msa_summ_df = base_df.groupby(['home_msa_name'])['tour_time'].agg(['mean', 'count'])

print 'Total East Suburban Mandatory Tours with project: {0:>18,.0f} tours'.format(base_msa_summ_df.loc['East Suburban']['count'])
print 'Total East Suburban Mandatory Tours without project: {0:>15,.0f} tours'.format(alt_msa_summ_df.loc['East Suburban']['count'])
print ''
print 'Average daily commute savings for an East Suburban resident: {0:>7,.2f} minutes'.format(alt_msa_summ_df.loc['East Suburban']['mean'] - base_msa_summ_df.loc['East Suburban']['mean'])
print 'Average annual commute savings for an East Suburban resident: {0:>6,.2f} minutes'.format((alt_msa_summ_df.loc['East Suburban']['mean'] - base_msa_summ_df.loc['East Suburban']['mean']) * 250)

Total East Suburban Mandatory Tours with project:            165,691 tours
Total East Suburban Mandatory Tours without project:         165,615 tours

Average daily commute savings for an East Suburban resident:    1.45 minutes
Average annual commute savings for an East Suburban resident: 363.20 minutes


**(East Suburban Residents) Get the Total Daily Time Saved with the Improvement**

In [40]:
daily_min_change = alt_df.loc[alt_df.home_msa == 3].tour_time.sum() - base_df.loc[base_df.home_msa == 3].tour_time.sum()
print '(E. Suburban Resident) Total Daily Change in Time Spent on Mandatory Tours: {0:>12,.1f} minutes'.format(daily_min_change)
print '(E. Suburban Resident) Total Daily Change in Time Spent on Mandatory Tours: {0:>12,.1f} hours'.format(daily_min_change / 60.)

(E. Suburban Resident) Total Daily Change in Time Spent on Mandatory Tours:    236,195.2 minutes
(E. Suburban Resident) Total Daily Change in Time Spent on Mandatory Tours:      3,936.6 hours


**(East Suburban Residents) Get the Total Annual Time Saved with the Improvement**

In [41]:
annual_min_change = daily_min_change * annualization_factor
print '(E. Suburban Resident) Total Annual Change in Time Spent on Mandatory Tours: {0:>15,.0f} minutes'.format(annual_min_change)
print '(E. Suburban Resident) Total Annual Change in Time Spent on Mandatory Tours: {0:>15,.0f} hours'.format(annual_min_change / 60.)

(E. Suburban Resident) Total Annual Change in Time Spent on Mandatory Tours:      59,048,806 minutes
(E. Suburban Resident) Total Annual Change in Time Spent on Mandatory Tours:         984,147 hours


**Monetary Equivalent of Lost Time Annually**

In [42]:
print '(E. Suburban Resident) Total Annual Commute Savings from Project: {0}'.format(locale.currency((annual_min_change / 60.) * value_of_time, grouping=True))

(E. Suburban Resident) Total Annual Commute Savings from Project: $12,301,834.56


**Change in User Fees (e.g., tolls, parking, transit fares) Associated without the Project**

In [44]:
annual_cost_difference = (alt_df.loc[alt_df.home_msa == 3].tour_cost.sum() - base_df.loc[base_df.home_msa == 3].tour_cost.sum()) * annualization_factor
print '(E. Suburban Resident) Total Annual Change in User Costs without the Project: {0}'.format(locale.currency(annual_cost_difference, grouping=True))

(E. Suburban Resident) Total Annual Change in User Costs without the Project: $294,652.50


**Change in Distance Travelled**

In [45]:
daily_change_pmt = alt_df.loc[alt_df.home_msa == 3].tour_distance.sum() - base_df.loc[base_df.home_msa == 3].tour_distance.sum()
print '(E. Suburban Resident) Total Daily Change in Person Miles Travelled on Mandatory Tour: {0:>16,.0f} miles'.format(daily_change_pmt)
print '(E. Suburban Resident) Total Annual Change in Person Miles Travelled on Mandatory Tour: {0:>15,.0f} miles'.format(daily_change_pmt * annualization_factor)

(E. Suburban Resident) Total Daily Change in Person Miles Travelled on Mandatory Tour:         -105,885 miles
(E. Suburban Resident) Total Annual Change in Person Miles Travelled on Mandatory Tour:     -26,471,249 miles
